# **Salomon Uran Parra C.C. 1015068767**

## **Laboratorio 5 ABC - Aprendizaje Estadistico**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### **1. Leer el data frame en formato csv en la dirección https://raw.githubusercontent.com/hernansalinas/Curso_aprendizaje_estadistico/main/datasets/Sesion_07_housing.csv**

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/hernansalinas/Curso_aprendizaje_estadistico/main/datasets/Sesion_07_housing.csv")
df.head()

### **2. Entender  el estado de los datos, para ello puedo emplear los comandos básicos del pandas**

  ```python
  df.info()
  df.describe()
  df.isnull().sum()
  df.isna().sum()
```


In [ ]:
df.info()

Hay una gran cantidad de datos NaN en la columna de total_bedrooms, por lo que se deberá plantear una estrategia para su limpieza o eliminación.

In [ ]:
#por ejemplo podemos eliminar los datos NaN
df.dropna(subset=["total_bedrooms"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

### **3. Determinar los elementos únicos dentro de la columna ocean_proximity.**

In [ ]:
print(df['ocean_proximity'].unique())
print(df.ocean_proximity.unique())

### **4. Para las columnas**


```python
cols = ["housing_median_age",	"total_rooms",	"total_bedrooms",	"population",	"households",	"median_income",	"median_house_value"]
```

Determinar el promedio de cada una de las columnas asociado a cada elementos unico de ocean_proximity, intenta con la operación groupby.

In [ ]:
cols = ["housing_median_age",   "total_rooms",  "total_bedrooms",   "population",   "households",   "median_income",    "median_house_value",'longitude','latitude']
df.groupby('ocean_proximity')[cols].mean()

### **5. Construye un histograma para cada columna, puede emplear la libreria de seaborn.**

In [ ]:
plt.figure(figsize=(20, 10))

#empleando las columnas del punto anterior
for i in cols:
  plt.subplot(3,4,cols.index(i)+1)
  sns.histplot(df, x = i, kde = True)

plt.show()

### **7. Empleando el siguiente código realiza el gráfico boxplot,**
```python
#draw boxplot
df.boxplot(column="median_house_value", by='ocean_proximity', sym = 'k.', figsize=(18,6))
#set title
plt.title('Boxplot for comparing price per living space for each city')
plt.show()
```

In [ ]:
#grafica el diagrama de caja del median_house_value para cada uno de los datos de ocean_proximity
df.boxplot(column="median_house_value", by='ocean_proximity', sym = 'k.', figsize=(18,6))

plt.title('Diagrama de caja de cercania al oceano versus valor medio de un hogar')
plt.show()

### **8. Determina la matrix de correlación.**

```python
corr_matrix = df.corr()
corr_matrix

plt.figure(figsize = (10,6))
sns.heatmap(corr_matrix, annot = True, cmap = "coolwarm", center=0)
plt.show()
```

In [ ]:
#primero crearemos un dataframe que contenga solo las columnas numericas del df
ndf = df.select_dtypes(include=[np.number])

corr_matrix = ndf.corr()
#se obtiene la matriz de correlacion

plt.figure(figsize = (10,6))
sns.heatmap(corr_matrix, annot = True, cmap = "coolwarm", center=0)
plt.show()

### **9. con las columnas, realiza un grafico pairplot empleando seaborn  de python.**
```python
cols = ["median_house_value", "median_income", "total_rooms","housing_median_age"]
```

In [ ]:
cols = ["median_house_value", "median_income", "total_rooms","housing_median_age"]
sns.pairplot(df[cols])
plt.show()

### **10. Realiza un scatter plot con la libreria sea born de python, el color del grafico puede ser empleado con la columna median_house_value**

In [ ]:
# Assuming 'df' is your pandas DataFrame
sns.scatterplot(data=df, x="median_income", y="housing_median_age", hue="median_house_value", palette="viridis")
plt.title('Scatter plot of Median Income vs Median House Value (colored by Median House Value)')
plt.show()

### **11. ¿Las siguiente linea es adecuada para separar el dataframe en datos de entrenamiento de test?, ¿que pasa en la división de los datos?**


```python
from sklearn.model_selection import train_test_split

# ¿Es significativa la muestra que se esta considerando?
train_set, test_set \
  = train_test_split(df, test_size = 0.2, random_state = 42)

print(len(train_set))
print(len(test_set))

```


In [ ]:
train_set, test_set \
  = train_test_split(df, test_size = 0.2, random_state = 42)

print(len(train_set))
print(len(test_set))

El anterior código está haciendo una separación aleatoria de los datos del df, y dedicando un 80% a entrenamiento del modelo y 20% a testeo de este. Esta estrategia es simple pero buena para generar remuestreos de los datos solo si los datos en el dataframe están distribuidos uniformemente y puedo asegurar que los nuevos muestreos conservan las proporciones y características del primero. Justamente esto último me permite asegurar que los conjuntos son representativos tanto para un entrenamiento del modelo como para un test del mismo.

### **12. División del dataset en grupos:**

La siguiente celda define un nuevo feature, income_cat, que divide en 5 categorias distintas los ingresos medios del dataframe.

In [ ]:
df["income_cat"] = pd.cut(df["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

display(df.head())
df.income_cat.hist()
plt.show()

Ahora, debido a que las categorías no tienen las mismas proporciones entre si (hay más o menos cantidad de una u otra), el remuestreo se hará basado en un shuffle (permutación) estratificado, de la siguiente forma:

In [ ]:
split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df, df["income_cat"]):
  strat_train_set = df.loc[train_index]
  strat_test_set = df.loc[test_index]

strat_train_set.reset_index(drop=True, inplace=True)
strat_test_set.reset_index(drop=True, inplace=True)

El código anterior hace los remuestreos del dataframe para obtener dos conjuntos de datos, un strat_train_set con 80% de los datos del df y un strat_test_set con el 20% restante. Teniendo en cuenta las proporciones (histograma) dadas por las categorías de ingresos medios que se definieron arriba, se debe cumplir que ambos conjuntos de datos nuevos poseen las mismas proporciones para cada categoría. También se comparará con el método anterior de sampleo aleatorio para el remuestreo.

In [ ]:
print(df["income_cat"].value_counts() / len(df))

print(strat_train_set["income_cat"].value_counts() / len(strat_train_set))

print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))

train_set, test_set \
  = train_test_split(df, test_size = 0.2, random_state = 42)

print(train_set["income_cat"].value_counts() / len(train_set))
print(test_set["income_cat"].value_counts() / len(test_set))

Del anterior print podemos ver que efectivamente, el shuffle estratificado funciona mucho mejor para preservar las proporciones del dataframe original en los nuevos conjuntos de entrenamiento y test, lo que los vuelve más representativos e ideales para dichos propósitos. El sampleo aleatorio en cambio, se aleja un poco de las proporciones del dataframe, como se puede observar en los últimos dos prints.

Por último, el siguiente código agrupa de forma clara los remuestreos aleatorios y estratificados para el conjunto de testeo, y crea dos métricas de errores porcentuales de los mismos respecto al dataframe original:


In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(df),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()

compare_props["Rand. %error"] =abs( 100 * compare_props["Random"] / compare_props["Overall"] - 100)
compare_props["Strat. %error"] =abs( 100 * compare_props["Stratified"] / compare_props["Overall"] - 100)

compare_props

Aquí podemos ver claramente la gran ventaja de usar el remuestreo estratificado, pues indica que los subconjuntos derivados del mismo son representativos y muy similares al original, a diferencia del remuestreo aleatorio.

### **13. Puedes agregar nuevas variables al dataframe para el análisis, por ejemplo:**
```python
df_train["rooms_per_household"] = df_train["total_rooms"]/df_train["households"]
df_train["bedrooms_per_room"] = df_train["total_bedrooms"]/df_train["total_rooms"]
df_train["population_per_household"]=df_train["population"]/df_train["households"]
```


In [ ]:
#ahora se definen nuevas variables en el df original
df["rooms_per_household"] = df["total_rooms"]/df["households"]
df["bedrooms_per_room"] = df["total_bedrooms"]/df["total_rooms"]
df["population_per_household"]=df["population"]/df["households"]
df.head()

### **14. Compara las siguientes variables:**
```python
imp_mean.statistics_
df_train_num.median()
```


```python
Constuye la matriz de características:

X = imp_mean.transform(df)
housing_tr = pd.DataFrame(X, columns=df_train_num.columns)
```

Sci-kit learn tiene una herramienta llamada imputer que sirve para rellenar o reemplazar los valores NaN o no validos del dataframe usando diferentes estrategias. A continuación vamos a usarlo para reemplazar los valores de total_bedrooms que son NaN.

In [ ]:
imp_mean = SimpleImputer(strategy="mean")
#se define el imputer usando sklearn.SimpleImputer y la estrategia mean, que es tomar la media de la columna como el valor a sustituir en el NaN

df1 = df.drop("ocean_proximity", axis=1)
#tomamos el dataframe sin la columna de la proximidad al oceano

imp_mean.fit(df1)
#hacemos el ajuste de las medias para el imputer

X = imp_mean.transform(df1)
#se transforma el dataframe reemplazando los datos NaN por las medias de sus columnas

df2 = pd.DataFrame(X, columns=df1.columns)
#redefinimos el dataframe como su transformacion por el imputer

#ahora procedamos a unirle la columna ocean proximity
df2 = df2.join(df["ocean_proximity"])
print(df2.info())


### **15.  ¿Qué realizan las siguientes lineas de código?**

```
from sklearn.preprocessing import OneHotEncoder
df_train["ocean_proximity"].unique()
housing_cat=df_train[["ocean_proximity"]]
housing_cat

cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
print(housing_cat_1hot)
print(cat_encoder.categories_)


df_cat_1hot = pd.DataFrame(housing_cat_1hot, columns = cat_encoder.categories_[0])

housing_tr_ = housing_tr.join(df_cat_1hot)
```


Como la columna ocean_proximity tiene datos no numéricos, no es ideal para entrenar un modelo de regresión con alguna columna como target. Por ello, se emplea la estrategia OneHotEncoder, que permite volver numéricas estas variables no numéricas o tipo string.

In [ ]:
strat_train_set["ocean_proximity"].unique()
housing_cat=strat_train_set[["ocean_proximity"]]

cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

display(housing_cat_1hot)
print(len(housing_cat_1hot))
print(cat_encoder.categories_)


df_cat_1hot = pd.DataFrame(housing_cat_1hot, columns = cat_encoder.categories_[0])

strat_train_set = strat_train_set.join(df_cat_1hot)

#vamos a eliminar la columna ocean_proximity
strat_train_set = strat_train_set.drop("ocean_proximity", axis=1)

strat_train_set

### **16. Las variables pueden ser escaladas como sigue:**

```python

cols=["longitude", "latitude",	"housing_median_age",	"total_rooms",\
      "total_bedrooms",	"population",	"households",	"median_income",\
      "<1H OCEAN",	"INLAND",	"ISLAND",	"NEAR BAY", "NEAR OCEAN"]


housing_scale=housing_tr_[cols]
housing_scale


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(housing_scale)

X = scaler.transform(housing_scale)


housing_prepared = pd.DataFrame(X, columns = housing_scale.columns)

```

Debido a que las diferentes variables asociadas a cada feature del dataframe poseen escalas y significados diferentes, esto puede representar un serio problema para la convergencia de algún método de optimización del modelo de regresión, por lo que es idóneo escalar las variables a intervalos que sean comparables entre sí. El siguiente código hace lo propio para las columnas indicadas, empleando el método MinMaxScaler que lo que hace es normalizar las columnas respecto a la diferencia entre su máximo y mínimo e inicializarlo en el dato mínimo, de tal forma que los datos van entre 0 y 1. Internamente se aplica la siguiente formula:

$$X_S = \frac{x-x_{\rm min}}{x_{\rm max} - x_{\rm min}}$$


In [ ]:
cols=["longitude", "latitude",  "housing_median_age",   "total_rooms",\
      "total_bedrooms", "population",   "households",   "median_income",\
      "<1H OCEAN",  "INLAND",   "ISLAND",   "NEAR BAY", "NEAR OCEAN"]


housing_scale = strat_train_set[cols]

housing_scale

scaler = MinMaxScaler()
scaler.fit(housing_scale)

X = scaler.transform(housing_scale)


housing_prepared = pd.DataFrame(X, columns = housing_scale.columns)

housing_prepared

### **17.** Para todos los pasos anteriores, contruye ordenadamente los pasos limpieza, escalamiento de variables, manejo de texto y atributos categóricos para tener el data frame listo para el análisis. Recuerda dividir el data frame en datos de entrenamiento y de test con la correcta estractificación. Genera dos data frame: housing_train, housing_test, cada una, debe tener las caracteristicas y los datos etiquetados.

Ahora que sabemos cómo realizar todo el procesado y preparación de los datos del dataframe, se puede realizar paso a paso la limpieza y preparación anterior de manera ordenada y lógica.

In [ ]:
#importemos nuevamente el dataframe
df = pd.read_csv("https://raw.githubusercontent.com/hernansalinas/Curso_aprendizaje_estadistico/main/datasets/Sesion_07_housing.csv")
df.info()

In [ ]:
#como ya sabemos que en los total_bedrooms tiene valores NaN, vamos a usar un imputer para reemplazar dichos datos por la media de la columna
#podemos reutilizar el codigo de antes pero ahora en vez de definir un nuevo dataframe df2, reemplazamos df por su transformacion

imp_mean = SimpleImputer(strategy="mean")
df1 = df.drop("ocean_proximity", axis=1)
imp_mean.fit(df1)
X = imp_mean.transform(df1)
df2 = pd.DataFrame(X, columns=df1.columns)
df = df2.join(df["ocean_proximity"])
print(df.info())

In [ ]:
#ahora, vamos a definir las nuevas variables y las categorias para los median_income
df["rooms_per_household"] = df["total_rooms"]/df["households"]
df["bedrooms_per_room"] = df["total_bedrooms"]/df["total_rooms"]
df["population_per_household"]=df["population"]/df["households"]
df["income_cat"] = pd.cut(df["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
df.head()

In [ ]:
#ahora podemos lidiar con el feature no numerico ocean_proximity, empleando la estrategia onehotencoder

df["ocean_proximity"].unique()
dftrans=df[["ocean_proximity"]]

cat_encoder = OneHotEncoder(sparse_output=False)
dftrans_1hot = cat_encoder.fit_transform(dftrans)

df_cat_1hot = pd.DataFrame(dftrans_1hot, columns = cat_encoder.categories_[0])

df = df.join(df_cat_1hot)

#vamos a eliminar la columna ocean_proximity
df = df.drop("ocean_proximity", axis=1)

df.head()


In [ ]:
#ahora, podemos escalar los features del dataframe de manera que sean comparables entre si
scaler = MinMaxScaler()
scaler.fit(df)
X = scaler.transform(df)
df = pd.DataFrame(X, columns = df.columns)
df["income_cat"] = df["income_cat"]*4 + 1
#revertimos el escalamiento para income_cat
df.head()

In [ ]:
#una vez preparado y listo el dataframe, podemos hacer un muestreo de datos de entrenamiento y test estratificado para entrenar un modelo multilineal

split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df, df["income_cat"]):
  housing_train = df.loc[train_index]
  housing_test= df.loc[test_index]

housing_train.reset_index(drop=True, inplace=True)
housing_test.reset_index(drop=True, inplace=True)

print(len(housing_train))
print(len(housing_test))

print(df["income_cat"].value_counts() / len(df))

print(housing_train["income_cat"].value_counts() / len(housing_train))

print(housing_test["income_cat"].value_counts() / len(housing_test))

#features y labels para el entreno y test

housing_train_features = housing_train.drop("median_house_value", axis=1)
housing_train_labels = housing_train["median_house_value"].copy()

print(len(housing_train_features))

print(len(housing_train_labels))

housing_test_features = housing_test.drop("median_house_value", axis=1)
housing_test_labels = housing_test["median_house_value"].copy()


#### **Preguntas adicionales del numeral 17**
1. ¿que puede concluir respecto al modelo empleado?
2. ¿El modelo de regresión lineal es valido para lo construido,
3. ¿qué informacion nos da el score?
4. ¿Puede ser ajustado a otro modelo?
5. ¿Como puede autmatizar todo el proceso empleando pipelines?

In [ ]:
#se crea el modelo lineal
lin_reg = LinearRegression()

#se hace un fiteo lineal con todos los features disponibles en el dataframe
lin_reg.fit(housing_train_features, housing_train_labels)

#se evalua el score en los features y labels del test
print(lin_reg.score(housing_test_features,housing_test_labels))

Debido a la compleja forma de los gráficos de dispersión para algunas columnas del dataframe, inicialmente no sería conveniente pensar que, por ejemplo, el valor medio de las casas pueda ser modelado con una dependencia lineal de alguna de las otras columnas. Sin embargo, aún es posible llevar a cabo dicha regresión. La celda anterior hace lo mencionado y calcula el score del modelo con los conjuntos de testeo definidos arriba. Como podemos observar, aun si incluimos todas las columnas o features del dataframe, el modelo lineal se comporta de manera muy pobre, pues un score de 0.65 indica que solo acierta un 65% de las veces. Esto es un poco más que adivinar el resultado, pero sigue siendo un desempeño muy pobre para las predicciones necesarias en el sector inmobiliario.

Muy seguramente, el que las gráficas de dispersión no tengan una forma definida significa que la correlación entre los datos tiene una estructura muy compleja, la cual un modelo lineal no es capaz de aproximar adecuadamente. Por ello se puede pensar mejor en redes neuronales u otro tipo de modelos de regresión, que sean mucho más robustos y complejos y que sean capaces de captar las estructuras del dataframe para una mejor predicción del valor medio de las casas.

Todo este proceso de limpieza, organización y preparación de los datos de entrenamiento y testeo, incluso junto con la construcción y evaluación del modelo, pueden ser realizados empleando pipelines. Los pipelines son procesos de manipulación y transformación de datos que consisten inicialmente en tres pasos: Origen de los datos, procesamiento o transformación de los datos, y destino o almacenamiento de estos. Son arquitecturas ordenadas que se encargan de realizar de manera coherente y eficiente los procedimientos de limpieza y preparación que hicimos anteriormente en el código y de forma muy separada. En este caso, podrían usarse librerías de pipelines robustas de Sci-Kit learn para haber realizado los procesamientos anteriores del dataframe.
